In [1]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/93/66/67cea8fd669c359af04502d5bdf00aa0bfe3918ee64280590a9c9f238c1f/boto3-1.28.83-py3-none-any.whl.metadata
  Using cached boto3-1.28.83-py3-none-any.whl.metadata (6.7 kB)
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/e2/3c/af9e6679c3743ded4feafd17a615215ac4dfe0e5255f3c6adae7f294ab81/awscli-1.29.83-py3-none-any.whl.metadata
  Using cached awscli-1.29.83-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/d1/79/07851f697697dc4fbc8b0daf3717e63b62ca54627d543e660991439de1dd/botocore-1.31.83-py3-none-any.whl.metadata
  Using cached botocore-1.31.83-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f

In [6]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [7]:
boto3_bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '01ee8d96-3148-40ac-b348-a2d49316826d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 09 Nov 2023 22:42:08 GMT',
   'content-type': 'application/json',
   'content-length': '6105',
   'connection': 'keep-alive',
   'x-amzn-requestid': '01ee8d96-3148-40ac-b348-a2d49316826d'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-g1-text-02',
   'modelId': 'amazon.titan-embed-g1-text-02',
   'modelName': 'Titan Text Embeddings v2',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'ou

In [8]:
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


In [26]:
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

In [42]:
def create_article(prompt):
    body = json.dumps({"prompt": prompt_data, 
                   "max_tokens_to_sample": 500,
                    "temperature": 1,
                    "top_k": 250,
                    "top_p": 1,
                    "stop_sequences": ["\n\nHuman:"]
                  })

    response = bedrock_runtime.invoke_model(body=body, 
                                          modelId=modelId, 
                                          accept=accept, 
                                          contentType=contentType)
    output_body = response.get('body')
    response_body = json.loads(response.get("body").read())

    output_body = response_body.get("completion")
    return output_body

In [43]:
prompt_data = """Human: As a news article writer, write me a 5 paragraph fictional news articles about {genre} with a title. Format the response as followed:

<title> title of the article </title>
<content> content of the arcticle </content>

Strictly follow the format described above. Only title and content are allowed in the response. Do not return anything else.

Assistant:"""

In [48]:
genres = ["politics", "sports", "entertainment", "science", "business"]
articles = []
for i in range(5):
    for genre in genres:
        prompt = prompt_data.replace("{genre}", genre)
        generated_article = create_article(prompt)
        articles.append(generated_article)

In [49]:
for i, article in enumerate(articles):
    with open(f"article-{i}.txt", "w") as f:
        f.write(article)